##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Generar rostros artificiales con el modelo GAN progresivo CelebA


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf_hub_generative_image_module"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/tf_hub_generative_image_module.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/tf_hub_generative_image_module.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/hub/tutorials/tf_hub_generative_image_module.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/google/progan-128/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelos de TF Hub</a>
</td>
</table>

En este Colab se enseña el uso de un módulo TF Hub basado en una red generativa adversativa (GAN por sus siglas en inglés). El módulo asigna vectores no dimensionales, llamados espacios latentes, a imágenes RGB.

Se proporcionan dos ejemplos:

- **Asignación** de espacios latentes a imágenes, y
- Dada una imagen de destino, **el uso del descenso de gradiente para encontrar** un vector latente que genere una imagen similar a la imagen de destino.

## Requisitos previos opcionales

- Familiaridad con los [conceptos de Tensorflow de bajo nivel](https://www.tensorflow.org/guide/eager).
- [Red generativa adversativa](https://en.wikipedia.org/wiki/Generative_adversarial_network) en Wikipedia.
- Artículo sobre GAN progresivas: [crecimiento progresivo de GAN para mejorar la calidad, la estabilidad y la variación](https://arxiv.org/abs/1710.10196).

### Más modelos

[Aquí](https://tfhub.dev/s?module-type=image-generator) puede encontrar todos los modelos hospedados actualmente en [tfhub.dev](https://tfhub.dev/) que pueden generar imágenes.

## Preparación

In [ ]:
# Install imageio for creating animations.  
!pip -q install imageio
!pip -q install scikit-image
!pip install git+https://github.com/tensorflow/docs

In [ ]:
#@title Imports and function definitions
from absl import logging

import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
tf.random.set_seed(0)

import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time

try:
  from google.colab import files
except ImportError:
  pass

from IPython import display
from skimage import transform

# We could retrieve this value from module.get_input_shapes() if we didn't know
# beforehand which module we will be using.
latent_dim = 512


# Interpolates between two vectors that are non-zero and don't both lie on a
# line going through origin. First normalizes v2 to have the same norm as v1. 
# Then interpolates between the two vectors on the hypersphere.
def interpolate_hypersphere(v1, v2, num_steps):
  v1_norm = tf.norm(v1)
  v2_norm = tf.norm(v2)
  v2_normalized = v2 * (v1_norm / v2_norm)

  vectors = []
  for step in range(num_steps):
    interpolated = v1 + (v2_normalized - v1) * step / (num_steps - 1)
    interpolated_norm = tf.norm(interpolated)
    interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
    vectors.append(interpolated_normalized)
  return tf.stack(vectors)

# Simple way to display an image.
def display_image(image):
  image = tf.constant(image)
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return PIL.Image.fromarray(image.numpy())

# Given a set of images, show an animation.
def animate(images):
  images = np.array(images)
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images)
  return embed.embed_file('./animation.gif')

logging.set_verbosity(logging.ERROR)

## Interpolación del espacio latente

### Vectores aleatorios

Interpolación del espacio latente entre dos vectores inicializados aleatoriamente. Usaremos un módulo TF Hub [progan-128](https://tfhub.dev/google/progan-128/1) que contiene una GAN progresiva preentrenada.

In [ ]:
progan = hub.load("https://tfhub.dev/google/progan-128/1").signatures['default']

In [ ]:
def interpolate_between_vectors():
  v1 = tf.random.normal([latent_dim])
  v2 = tf.random.normal([latent_dim])
    
  # Creates a tensor with 25 steps of interpolation between v1 and v2.
  vectors = interpolate_hypersphere(v1, v2, 50)

  # Uses module to generate images from the latent space.
  interpolated_images = progan(vectors)['default']

  return interpolated_images

interpolated_images = interpolate_between_vectors()
animate(interpolated_images)

## Encontrar el vector más cercano en el espacio latente

Corriga una imagen de destino. Como ejemplo, use una imagen generada a partir del módulo o cargue una propia.

In [ ]:
image_from_module_space = True  # @param { isTemplate:true, type:"boolean" }

def get_module_space_image():
  vector = tf.random.normal([1, latent_dim])
  images = progan(vector)['default'][0]
  return images

def upload_image():
  uploaded = files.upload()
  image = imageio.imread(uploaded[list(uploaded.keys())[0]])
  return transform.resize(image, [128, 128])

if image_from_module_space:
  target_image = get_module_space_image()
else:
  target_image = upload_image()

display_image(target_image)

Después de definir una función de pérdida entre la imagen de destino y la imagen generada por una variable de espacio latente, podemos usar el descenso de gradiente para encontrar valores de variables que minimicen la pérdida.

In [ ]:
tf.random.set_seed(42)
initial_vector = tf.random.normal([1, latent_dim])

In [ ]:
display_image(progan(initial_vector)['default'][0])

In [ ]:
def find_closest_latent_vector(initial_vector, num_optimization_steps,
                               steps_per_image):
  images = []
  losses = []

  vector = tf.Variable(initial_vector)  
  optimizer = tf.optimizers.Adam(learning_rate=0.01)
  loss_fn = tf.losses.MeanAbsoluteError(reduction="sum")

  for step in range(num_optimization_steps):
    if (step % 100)==0:
      print()
    print('.', end='')
    with tf.GradientTape() as tape:
      image = progan(vector.read_value())['default'][0]
      if (step % steps_per_image) == 0:
        images.append(image.numpy())
      target_image_difference = loss_fn(image, target_image[:,:,:3])
      # The latent vectors were sampled from a normal distribution. We can get
      # more realistic images if we regularize the length of the latent vector to 
      # the average length of vector from this distribution.
      regularizer = tf.abs(tf.norm(vector) - np.sqrt(latent_dim))
      
      loss = target_image_difference + regularizer
      losses.append(loss.numpy())
    grads = tape.gradient(loss, [vector])
    optimizer.apply_gradients(zip(grads, [vector]))
    
  return images, losses


num_optimization_steps=200
steps_per_image=5
images, loss = find_closest_latent_vector(initial_vector, num_optimization_steps, steps_per_image)

In [ ]:
plt.plot(loss)
plt.ylim([0,max(plt.ylim())])

In [ ]:
animate(np.stack(images))

Compare el resultado con el destino:

In [ ]:
display_image(np.concatenate([images[-1], target_image], axis=1))

### Jugar con el ejemplo anterior

Si la imagen proviene del espacio del módulo, el descenso es rápido y converge a una muestra razonable. Intente descender a una imagen que **no sea del espacio del módulo**. El descenso solo convergerá si la imagen está razonablemente cerca del espacio de las imágenes de entrenamiento.

¿Cómo hacer que descienda más rápido y a una imagen más realista? Se puede intentar:

- mediante el uso de diferentes pérdidas en la diferencia de imagen, por ejemplo, cuadrática,
- mediante el uso de diferentes regularizadores en el vector latente,
- inicializar desde un vector aleatorio en múltiples ejecuciones,
- etc.
